# Análisis de MNIST con tf.keras, tf.data y eager execution

**Profesor:** Roberto Muñoz <br />
**E-mail:** <rmunoz@metricarts.com> <br />

**Colaborador:** Sebastián Arpón <br />
**E-mail:** <rmunoz@metricarts.com> <br />

En este laboratorio crearemos una red neuronal que pueda detectar a que digito corresponde una imagen que recibe (note que cada imagen contendra solo un digito). Utilizaremos la API `tf.data` [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) la cual es muy eficiente e incluye funcionalidades como el shuffling y batching. 

El conjunto de datos con el que trabajaremos es el MINST el cual, como veremos mas adelante esta incluido en KERAS.

In [ ]:
import tensorflow as tf

# Activando Eager
tf.enable_eager_execution()

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
tf.__version__

In [ ]:
# obteniendo la data
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

## Revisamos el tamaño de train_images y train_labels

In [ ]:
print(type(train_images))

In [ ]:
print(len(train_images))
print(train_images.shape)

In [ ]:
print(type(train_labels))

In [ ]:
print(len(train_labels))
print(train_labels.shape)

## Revisamos un par de imágenes del dataset train

In [ ]:
i=np.random.randint(len(train_images))

print("Indice del registro: ", i)
print("Label: ", train_labels[i])
plt.imshow(train_images[i,:,:])

## Transformamos train_images y test_images de 28x28 a 784

In [ ]:
# Chequeando el tamaño de los conjuntos de entrenamiento y test
TRAINING_SIZE = len(train_images)
TEST_SIZE = len(test_images)

# transformando desde (N, 28, 28) a (N, 784)
train_images = np.reshape(train_images, (TRAINING_SIZE, 784))
test_images = np.reshape(test_images, (TEST_SIZE, 784))

# Transformando cada arreglo desde uint8 a float32
train_images = train_images.astype(np.float32)
test_images = test_images.astype(np.float32)
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

# Convirtiendo cada valor desde [0,255] a [0,1] 
train_images /= 255
test_images /=  255

In [ ]:
NUM_DIGITS = 10

print("Previo al cambio de formato", train_labels[0]) # The format of the labels before conversion

train_labels  = tf.keras.utils.to_categorical(train_labels, NUM_DIGITS)

print("Posterior al cambio de formato", train_labels[0]) # The format of the labels after conversion

test_labels = tf.keras.utils.to_categorical(test_labels, NUM_DIGITS)

In [ ]:
# Creando la red neuronal con KERAS

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))


# Creando el optimizador 
# Esto es obligatorio mientras usamos eager execution
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)

# Ahora compilaremos y mostraremos el resultado de nuestro modelo
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

### Step 1) Creando un tf.data Dataset

Ahora usaremos `tf.data.Dataset` [API](https://www.tensorflow.org/api_docs/python/tf/data) para convertir los arreglos de Numpy en un dataset de TensorFlow

A continuacion crearemos un ciclo **for** que servira como una introduccion en la creacion de ciclos de entrenamientos personalizados. Aunque esencialmente estos ciclos hacer lo mismo que `model.fit`, esto nos permite personalizar todo el proceso y recolectar distintas metricas.

In [ ]:
BATCH_SIZE=128

# Dado que tf.data puede funcionar en colecciones de datos potencialmente grandes
# La desordenaremos por partes.
SHUFFLE_SIZE = 10000 

# Creando el dataset
dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
dataset = dataset.shuffle(SHUFFLE_SIZE)
dataset = dataset.batch(BATCH_SIZE)

### Step 2) Iterando sobre el dataset
Aca entrenaremos sobre el dataset usando los distintos batch.

In [ ]:
EPOCHS=5

for epoch in range(EPOCHS):
    for images, labels in dataset:
        print(type(images), type(labels))
        train_loss, train_accuracy = model.train_on_batch(images, labels)
  
  # Obtenemos cualquier metrica o ajustamos los parametros de entrenamiento
    print('Epoch #%d\t Loss: %.6f\tAccuracy: %.6f' % (epoch + 1, train_loss, train_accuracy))
  

In [ ]:
loss, accuracy = model.evaluate(test_images, test_labels)
print('Test accuracy: %.2f' % (accuracy))